In [49]:
eg = :[2*a*[1 2] for a in 1:1]

:($(Expr(:comprehension, :(2 * a * [1 2]), :(a = 1:1))))

In [44]:
eg.head

:comprehension

In [45]:
eg.args[2].args

2-element Array{Any,1}:
 :a    
 :(1:4)

In [108]:
macro reg(comp)
    @assert(comp.head==:comprehension)
    #@assert(length(comp.args)==2) #Don't want to do things that are already 2D
    raw = eval(comp)
    
    if length(raw)==0
        return raw
        
    elseif length(raw)==1
        return raw[1]
    else
        if (typeof(raw[1])==typeof(raw[2]) <: AbstractArray) && size(raw[1])==size(raw[2])
            cat_dim = indmin(size(raw[1]))
            return cat(cat_dim, raw...)
        end
        
    end
    
end

In [109]:
@reg [[1 2] for a in 1:4]

4x2 Array{Int64,2}:
 1  2
 1  2
 1  2
 1  2

In [105]:
@reg [[1, 2] for a in 1:4]

2x4 Array{Int64,2}:
 1  1  1  1
 2  2  2  2

In [110]:
@reg [[1; 2] for a in 1:4]

8-element Array{Int64,1}:
 1
 2
 1
 2
 1
 2
 1
 2

In [111]:
@reg,[[1 2] for a in 1:4]

4x2 Array{Int64,2}:
 1  2
 1  2
 1  2
 1  2

In [118]:
dump(:(a|>b|>c))

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol |>
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol |>
        2: Symbol a
        3: Symbol b
      typ: Any
    3: Symbol c
  typ: Any


In [59]:
data = [2.255, 3.755, 6.888, 7.999, 9.001]
log(2,sum(round(data,2)))


4.9025560053446275

In [277]:
@pipe data|>round(_,2)|>sum|>log(2,_)

4.9025560053446275

In [60]:
data |>   x -> round(x,2) |> sum |> x -> log(2,x)

4.9025560053446275

In [221]:
#No change to nonpipes functionality 
@assert macroexpand( :(@pipe a) ) == :a #doesn't change single inputs
@assert macroexpand( :(@pipe b(a)) ) == :(b(a)) #doesn't change inputs that a function applications

#Compatable with Julia 1.3 piping functionality
@assert macroexpand( :(@pipe a|>b) ) == :(b(a)) #basic
@assert macroexpand( :(@pipe a|>b|>c) ) == :(c(b(a)))  #Keeps chaining 3
@assert macroexpand( :(@pipe a|>b|>c|>d) ) == :(d(c(b(a)))) #Keeps chaining 4

@assert macroexpand( :(@pipe a|>b(x)) ) == :((b(x))(a))  #applying to function calls returning functions
@assert macroexpand( :(@pipe a(x)|>b ) ) == :(b(a(x)))   #feeding functioncall results on wards

@assert macroexpand(:(@pipe 1|>a)) ==:(a(1)) #Works with literals (int)
@assert macroexpand(:(@pipe "foo"|>a)) == :(a("foo")) #Works with literal (string)
@assert macroexpand( :(@pipe a|>bb[2])) == :((bb[2])(a)) #Should work with RHS that is a array reference


#Marked locations
@assert macroexpand( :(@pipe a|>b(_) ) ) == :(b(a)) #Marked location only
@assert macroexpand( :(@pipe a|>b(x,_) ) ) == :(b(x,a)) # marked 2nd (and last)
@assert macroexpand( :(@pipe a|>b(_,x) ) ) == :(b(a,x)) # marked first
@assert macroexpand( :(@pipe a|>b(_,_) ) ) == :(b(a,a)) # marked double (Not certain if this is a good idea)
@assert macroexpand( :(@pipe a|>bb[2](x,_))) == :((bb[2])(x,a)) #Should work with RHS that is a array reference


#Applying things to the mark
@assert macroexpand( :(@pipe a|>bb(_[1]))) == :(bb(a[1])) #Should work with indexing off of the subsitition
@assert macroexpand( :(@pipe a|>bb[2:3](_[1]))) == :((bb[2:3])(a[1])) #Should work with indexing off of the subsitition off of a complex experession
@assert macroexpand( :(@pipe a|>bb(_(1,2)))) == :(bb(a(1,2))) #Should work with calling off of the subsitition
@assert macroexpand( :(@pipe a|>b(_...) ) ) == :(b(a...)) # Unpacking
@assert macroexpand( :(@pipe a|>bb[2](_...))) == :((bb[2])(a...)) #Should work with RHS of arry ref and do unpacking

#Mixing modes
@assert macroexpand( :(@pipe a|>b|>c(_) ) ) == :(c(b(a)))
@assert macroexpand( :(@pipe a|>b(x,_)|>c|>d(_,y) ) ) == :(d(c(b(x,a)),y))
@assert macroexpand( :(@pipe a|>b(xb,_)|>c|>d(_,xd)|>e(xe) |>f(xf,_,yf) ) ) == :(f(xf,(e(xe))(d(c(b(xb,a)),xd)),yf)) #Very Complex

In [195]:
#Reflow piped things replacing the _ in the next section

function rewrite(ff::Expr,target)
    function replace(arg::Any)
        arg #Normally do nothing
    end
    function replace(arg::Symbol)
        if arg==:_
            target
        else
            arg
        end
    end
    function replace(arg::Expr)
        rep = copy(arg)
        rep.args = map(replace,rep.args)
        rep
    end
       
    if (ff.head==:call)
        rep_args = map(replace,ff.args)
        if ff.args != rep_args
            #_ subsitution
            ff.args=rep_args
            return ff
        end
    end
    #No subsitution was done (either cos not a call, or cost no _ found)
    #Apply to a function that is being returned by ff, (ff could be a function call or something more complex)
    rewrite_apply(ff,target)
end
        

function rewrite_apply(ff::Union(Symbol,Expr),target)
    #function application
    :($ff($target))
end

function rewrite(ff::Symbol,target) 
    rewrite_apply(ff,target)
end

function funnel(ee::Any) #Could be a Symbol could be a literal
    #first (left most) input
    ee
end

function funnel(ee::Expr)
    if (ee.args[1]==:|>)
        ff = ee.args[3]
        target = funnel(ee.args[2]) #Recurse
        
        rewrite(ff,target)
    else
        #Not in a piping situtation
        ee #make no change
    end
end

macro pipe(ee)
    funnel(ee)    
end

In [196]:
macroexpand(:(@pipe a |> (bb[2])(x,_)))

:((bb[2])(x,a))

In [220]:
macroexpand( :(@pipe a|>bb(_(1,2)))) == :(bb(a(1,2))) #Should work with calling off of the subsitition


true

In [249]:
function ratio(value, lr, rr)
    println("slitting on ration $lr:$rr")
    value*lr/(lr+rr), value*rr/(lr+rr)
end

function percent(left, right)
    left/right*100
end

@pipe 10 |> ratio(_,4,1) |> percent(_[1],_[2]) # = 400.0, outputs slitting on ration 4:1 Twice
@pipe 10 |> ratio(_,4,1) |> percent(_...) # = 400.0, outputs slitting on ration 4:1 Once




slitting on ration 4:1
slitting on ration 4:1
slitting on ration 4:1


400.0

In [234]:
tan(pi/4)

0.9999999999999999

In [116]:
(a|>bb[2] ).args[3].ar

2-element Array{Any,1}:
  :bb
 2   

In [117]:
dump(:(a|>bb[2]))

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol |>
    2: Symbol a
    3: Expr 
      head: Symbol ref
      args: Array(Any,(2,))
        1: Symbol bb
        2: Int64 2
      typ: Any
  typ: Any


In [82]:
dump(:(b(_[1](x))).args)

Array(Any,(2,))
  1: Symbol b
  2: Expr 
    head: Symbol call
    args: Array(Any,(2,))
      1: Expr 
        head: Symbol ref
        args: Array(Any,(2,))
          1: Symbol _
          2: Int64 1
        typ: Any
      2: Symbol x
    typ: Any


In [85]:
:(a(s)).args <:Vector{Any}


LoadError: type: subtype: expected Type{T<:Top}, got Array{Any,1}
while loading In[85], in expression starting on line 1